In [5]:
#全局变量
from dateutil.parser import parse
import numpy as np
import pandas as pd
filep='D:\\python-document\\rerun\\'
#经常用的函数

def trans_time(f):
    f1=f.apply(lambda x:parse(x))
    f1=f1.apply(lambda x:str(x.year)+'-'+'{:02}'.format(x.month))
    return f1
def to_time(f):
    f1=f.apply(lambda x:parse(x))
    return f1
import re
def month_differ(x, y):
    """暂不考虑day, 只根据month和year计算相差月份
    Parameters
    ----------
    x, y: 两个datetime.datetime类型的变量
 
    Return
    ------
    differ: x, y相差的月份
    """
    xyear=np.array([i.year for i in x])
    yyear=np.array([i.year for i in y])
    xmonth=np.array([i.month for i in x])
    ymonth=np.array([i.month for i in y])
    month_differ = (xyear - yyear) * 12 + (xmonth - ymonth) * 1
    return month_differ
import random

# 数据合并

## 数据库导出（评论数据）

In [62]:
#将sql中的数据导入到dataframe中
import pandas as pd
from sqlalchemy import create_engine
db_info = {'user': 'root',
            'password': '267708',
            'host': 'localhost',
           'port': 3306,
            'database': 'spider'
           }

engine = create_engine('mysql+pymysql://%(user)s:%(password)s@%(host)s:%(port)d/%(database)s?charset=utf8' % db_info, encoding='utf-8')
df = pd.read_sql_table('ta_commentfinal', engine)
#df.to_csv("d://python-document//tripadviser.csv")
#df.info()
df.isnull().sum()
#填充空值
nacolumn_zero=['user_name','user_votes','comment_thanks','comment_topic','user_review_counts',
               'user_attrction_review_counts','picture_count','visited_time']
nacolumn_one=['user_level']
for i in nacolumn_zero:
    df[i]=df[i].fillna(0)
for i in nacolumn_one:
    df[i]=df[i].fillna(1)
df.info()
'''
for i in df.columns:
    df[i]=df[i].fillna(0)
df.info()
'''
#将user_level换成整数
print(df.user_level.value_counts())
df.user_level=df.user_level.apply(lambda x: int(float(x)))
df.user_level[:10]
print(df.user_level.value_counts())
#将location空格去掉
df.user_location=df.user_location.apply(lambda x:x.strip())#只能运行一遍
df.head()
print(sum(df.duplicated()))#是否有重复值
df['review_time']=df['review_time'].apply(lambda x:x.strip())#去除不必要的空格
#处理visited_time
import re
df['visited_time']=df['visited_time'].apply(lambda x:str(x).strip())#去除不必要的空格
df['visited_time']=df['visited_time'].apply(lambda x:re.search('Visited (.*)',x)[1] if x!='0' else '0')
#df['visited_time'][:4]   
df.to_csv("%s1.1com_origin.csv"%filep,index=False)

F:\Adaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xD6\\xD0\\xB9\\xFA\\xB1\\xEA...' for column 'VARIABLE_VALUE' at row 496")
  result = self._query(query)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182097 entries, 0 to 182096
Data columns (total 15 columns):
attraction_id                   182097 non-null object
comment_id                      182097 non-null object
user_name                       182097 non-null object
user_location                   182097 non-null object
user_level                      182097 non-null object
user_review_counts              182097 non-null object
user_attrction_review_counts    182097 non-null object
user_votes                      182097 non-null object
review_time                     182097 non-null object
score                           182097 non-null object
comment_topic                   182097 non-null object
comment_content                 182097 non-null object
comment_thanks                  182097 non-null object
visited_time                    182097 non-null object
picture_count                   182097 non-null object
dtypes: object(15)
memory usage: 20.8+ MB


'\nfor i in df.columns:\n    df[i]=df[i].fillna(0)\ndf.info()\n'

In [101]:
2019-04-19 23:26:49 2019-04-19 23:26:50 

F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


171631
171631


F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [106]:
df['comment_content_count']=df['comment_content'].apply(lambda x:len([i for i in x.strip().split(' ')]))
df['comment_counts']=1
df_groupby=df.groupby(['attraction_id',"review_time"])['user_level', 'user_review_counts', 'user_attrction_review_counts',
'user_votes', 'score','comment_thanks','comment_content_count','picture_count'].mean().reset_index()
comment_count=df.groupby(['attraction_id',"review_time"])['comment_counts'].sum().reset_index()
#comment_count=data.groupby(['attraction_id',"review_time"]).agg('comment_content_count',count)
df_groupby=pd.merge(df_groupby,comment_count,on=['attraction_id',"review_time"],how='left')
#取小数
df_groupby['attraction_id']=df_groupby['attraction_id'].apply(lambda x:str(x))
for i in df_groupby.columns[2:]:
    df_groupby[i]= df_groupby[i].apply(lambda x:round(x,2))
df_groupby.to_csv("%s1.1com_bymonth.csv"%filep,index=False)

In [107]:
df_groupby.duplicated().sum()

0

## 整合景点数据

In [109]:
#景点主页数据
attraction=pd.read_excel(u'F:\\研究生毕设数据\\divvybike data\\data.xlsx',sheet_name='all')
attraction.dropna(subset=['latitude'],inplace=True)#去掉坐标为空的景点
attraction.to_csv("%s1.2attraction.csv"%filep,index=False)

#将评论数据与景点的位置联系起来
attaction=pd.read_csv("%s1.2attraction.csv"%filep)
attaction.shape
tripadviser=pd.read_csv("%s1.1com_bymonth.csv"%filep)
trip_attraction=pd.merge(tripadviser,attaction,left_on='attraction_id',right_on='id',how='left')
#trip_attraction.dropna(subset=['latitude'],inplace=True)#去掉坐标为空的景点
#去除坐标值为空的数据
trip_attraction.dropna(subset=['latitude'],inplace=True)#去掉坐标为空的景点
trip_attraction.info()
trip_attraction.to_csv("%s1.2comment_attraction.csv"%filep,index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20391 entries, 0 to 20663
Data columns (total 25 columns):
attraction_id                   20391 non-null int64
review_time                     20391 non-null object
user_level                      20391 non-null float64
user_review_counts              20391 non-null float64
user_attrction_review_counts    20391 non-null float64
user_votes                      20391 non-null float64
score                           20391 non-null float64
comment_thanks                  20391 non-null float64
comment_content_count           20391 non-null float64
picture_count                   20391 non-null float64
comment_counts                  20391 non-null int64
id                              20391 non-null float64
name                            20391 non-null object
walk score                      20391 non-null float64
transit score                   20391 non-null float64
bike score                      20391 non-null float64
types            

## 整合站点数据（按距离）

### 站点数据

In [113]:
import pandas as pd
station=pd.read_csv(u'F:\\研究生毕设数据\\divvybike data\\Divvy_Trips_2017_Q3Q4\\Divvy_Stations_2017_Q3Q4.csv',
                    encoding='utf-8',engine='python')
#station.drop(station.columns[-1],axis=1,inplace=True)#去掉最后的空列
station['online_date1']=trans_time(station['online_date'])
station.to_csv("%s1.3.1station.csv"%filep,index=False)
station['online_date1'].value_counts()#主要集中于2013,2015,2016

2013-06    86
2013-07    85
2016-05    80
2015-02    79
2013-09    79
2013-08    67
2015-01    61
2016-06    23
2013-10    13
2013-12     2
2016-07     2
2014-01     1
2017-08     1
2017-12     1
2017-04     1
2014-04     1
2014-03     1
2017-06     1
2017-07     1
Name: online_date1, dtype: int64

### 最短距离

In [120]:
#然后计算与station在对应时间的最近位置
import pandas as pd
from geopy.distance import geodesic#可以简便计算位置的包，包含很多与地理数据处理有关的方法
trip_attraction=pd.read_csv("%s1.2comment_attraction.csv"%filep)
station=pd.read_csv("%s1.3.1station.csv"%filep)

#最短距离及对应站点
final=[]
for i in trip_attraction.index: 
#    print(trip_attraction.loc[i].values[2])
    stationslice=station[station['online_date1']<=trip_attraction['review_time'][i]]
    distance={}
    a=trip_attraction.loc[i].values[-2]
    b=trip_attraction.loc[i].values[-1]
    for x,y,z in zip(stationslice['latitude'].values,stationslice['longitude'].values,stationslice['id'].values):
        dis=geodesic((a,b), (x,y)).m
        distance[z]=dis
    if len(distance)==0:
        final.append(('null','null'))
    else:
        final.append(min(distance.items(), key=lambda x: x[1]))
print(len(final))
#然后将最短距离字典进行整合
final1=pd.DataFrame(final,columns=['recent_station','shortest_dis'])
trip_attraction1=pd.concat([trip_attraction,final1],axis=1)
trip_attraction1['shortest_dis']=trip_attraction1['shortest_dis'].apply(lambda x:0 if x=='null' else x)
trip_attraction1['recent_station']=trip_attraction1['recent_station'].apply(lambda x:0 if x=='null' else x)
#trip_attraction1['shortest_dis']=trip_attraction1['shortest_dis'].fillna(100000).apply(lambda x:round(float(x),0))
trip_attraction1.to_csv("%s1.3.2shortest_dis.csv"%filep)
print(trip_attraction1.shape)#(20391, 27)
print(sum(trip_attraction1['shortest_dis'].values>=800))#585   600
print(sum(trip_attraction1['shortest_dis'].values>=600))#729  749

20391
(20391, 27)
600
749


### 距离中间结果

In [123]:
#做200m，300m等
from geopy.distance import geodesic#可以简便计算位置的包，包含很多与地理数据处理有关的方法
trip_attraction=pd.read_csv("%s1.3.2shortest_dis.csv"%filep)
station=pd.read_csv("%s1.3.1station.csv"%filep)

#计算中间结果
final=[]
for i in trip_attraction.index: 
    stationslice=station[station['online_date1']<=trip_attraction.loc[i]['review_time']]
    a=trip_attraction.loc[i]['latitude']
    b=trip_attraction.loc[i]['longitude']
    if not stationslice.empty:
        jieguozj=[]
        for x,y,z,d in zip(stationslice['latitude'].values,stationslice['longitude'].values,stationslice['id'].values,
                   stationslice['online_date1'] ):
            dis=geodesic((a,b), (x,y)).m
            jieguozj.append([trip_attraction['attraction_id'][i],trip_attraction['review_time'][i],z,d,dis])
        final.append(jieguozj)
print(len(final))
#保存中间结果并分析，将其整理到原来的表中
#保存中间结果，做好备份
for item in final:
    with open("%s1.3.3distance_of_station_attraction.txt"%filep,'a') as f:
        f.write(str(item)+'\n')

16150


### 具体距离（200m等）

In [124]:
#读取结果并分析
import pandas as pd
def read_mjieguo(f):
    with open(f,'r') as f:
        for i in f:
            yield i
f="%s1.3.3distance_of_station_attraction.txt"%filep
def calcD(d=0):
    huizong=[]
    for i in read_mjieguo(f):
        ad=pd.DataFrame(eval(i),columns=['attraction_id','review_time','station_id','station_onlinetime','distance'])
        ad['distance']=ad['distance'].apply(lambda x:int(round(float(x),0)))
        ads=ad[ad.distance<=d]
        if not ads.empty:
            alist=[list(ads.attraction_id)[0],list(ads.review_time)[0],list(ads['station_id']),
                   list(ads['station_onlinetime']),list(ads['distance'])]#不一定能被赋值
            #赋值之后变成了局部变量，搞不懂
            huizong.append(alist)
    huizong1=pd.DataFrame(huizong,
                          columns=['attraction_id','review_time','station_id','station_onlinetime','distance']) 
    huizong1.to_csv("%s1.3.4_"%filep+str(d)+"m.csv",index=False)
calcD(d=200)
calcD(d=300)

In [125]:
#一并整合进去，汇总数据
import pandas as pd
def station_d(d):
    tripad_attraction_station=pd.read_csv("%s1.3.2shortest_dis.csv"%filep)
    t_slice=tripad_attraction_station[['attraction_id','review_time']]
    datax=pd.read_csv("%s1.3.4_"%filep+d+"m.csv")
    datax['station_time_'+d]=datax.station_onlinetime.apply(lambda x:min(eval(x)))
    datax['has_station_'+d]=datax['station_time_'+d].apply(lambda x:1 if x!='0' else 0) 
    datax['station_count_'+d]=datax.station_id.apply(lambda x:len(eval(x)))
    data_merge=pd.merge(t_slice,datax,on=['attraction_id','review_time'],how='left')
    data_merge.drop(['station_id','station_onlinetime','distance'],axis=1,inplace=True)
    data_merge[['station_time_'+d,'has_station_'+d,'station_count_'+d]]=data_merge[['station_time_'+d,'has_station_'+d,'station_count_'+d]].fillna(0)
    data_merge1=pd.merge(tripad_attraction_station,data_merge,on=['attraction_id','review_time'],how='left')
    data_merge1=data_merge1.drop_duplicates(['attraction_id','review_time'])
    data_merge1.to_csv("%s1.3.4_..+station"%filep+d+'m.csv',index=0)
    return data_merge1
station_d('200')
station_d('300')

,Unnamed: 0,attraction_id,review_time,user_level,user_review_counts,user_attrction_review_counts,user_votes,score,comment_thanks,comment_content_count,...,group,address,near,latitude,longitude,recent_station,shortest_dis,station_time_300,has_station_300,station_count_300
0,0,103238,2002-02,1.00,0.00,0.00,0.00,4.00,18.00,80.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
1,1,103238,2004-01,1.00,1.00,0.00,8.00,5.00,8.00,21.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
2,2,103238,2004-03,1.00,1.00,0.00,5.00,3.00,5.00,47.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
3,3,103238,2004-04,1.00,1.00,0.00,9.00,5.00,9.00,67.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
4,4,103238,2004-09,2.00,9.00,5.00,37.00,5.00,14.00,40.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
5,5,103238,2004-10,1.00,1.00,0.00,17.00,5.00,17.00,71.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
6,6,103238,2004-12,4.00,24.00,9.50,272.50,3.50,22.50,141.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
7,7,103238,2005-09,6.00,185.00,35.00,300.00,4.00,12.00,124.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
8,8,103238,2006-01,5.00,52.00,10.00,71.00,1.00,35.00,387.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
9,9,103238,2006-05,4.00,31.00,8.00,103.00,4.00,10.00,84.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0


## 整合divvy_trips数据（按距离）

In [4]:
import pandas as pd
#读取文件名
import os
f='D:\\python-document\\divvy_trips\\xx\\'
def trips_origin(f):
    filelist=os.listdir(f)
    col=['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',
         'from_station_id', 'from_station_name', 'to_station_id',
         'to_station_name', 'usertype', 'gender', 'birthyear','trip_time','is_sub',
         'is_male','is_female','age']
    huizongz=pd.DataFrame(columns=col)
    for item in filelist:
        medi=pd.read_csv(f+item)
        medi.columns=['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthyear']
        medi['trip_time']=medi['stoptime']
        medi['trip_time']=to_time(medi['trip_time'])
        try:
            medi['tripduration']=medi['tripduration'].apply(lambda x: int(float(x.replace(',',''))))#这个是有字符串，的版本
        except:
            medi['tripduration']=medi['tripduration']
        medi['is_sub']=medi['usertype'].apply(lambda x:1 if x=='Subscriber' else 0)
        medi['is_male']=medi['gender'].apply(lambda x:1 if x=='Male' else 0)
        medi['is_female']=medi['gender'].apply(lambda x:1 if x=='Female' else 0)
        medi['age']=medi['trip_time'].apply(lambda x:x.year)-medi['birthyear']
        huizongz=pd.concat([huizongz,medi],axis=0)
        print(item,':完成')

#将非订阅用户的用户数据去掉，数据太少，不利于之后的分析
    huizongz['is_male'][huizongz.is_sub==0]=0
    huizongz['is_female'][huizongz.is_sub==0]=0
    huizongz['age'][huizongz.age>90.0]=90.0
    huizongz['age'][huizongz.age<10.0]=10.0
    huizongz['age'][huizongz.is_sub==0]=0


    huizongz['tripduration'][huizongz.tripduration>12*3600]=12*3600
    huizongz['is_male'][(huizongz.is_sub==1) & (huizongz.gender.isnull())]=huizongz['is_male'][(huizongz.is_sub==1) & (huizongz.gender.isnull())]\
    .apply(lambda x:random.choices([1,0], weights=[0.75,0.25],k=1)[0])
    huizongz['is_female'][(huizongz.is_sub==1) & (huizongz.gender.isnull())]=1-huizongz['is_male']\
    [(huizongz.is_sub==1) & (huizongz.gender.isnull())].values
    print('finish')
    huizongz['trip_time']=trans_time(huizongz['trip_time'])    
    huizongz.to_csv("%s1.4.0trips_origin.csv"%filep,index=0)
    return huizongz

'''
#没有区分入站和出站点
import pandas as pd
#读取文件名
import os
filelist=os.listdir('D:\\python-document\\divvy_trips\\xx\\')
a='D:\\python-document\\divvy_trips\\xx\\'
aad=pd.read_csv('D:\\python-document\\tripad_attraction_station_(400,600,800).csv')
dataq=aad[['attraction_id','review_time','recent_station']]
huizongz=pd.DataFrame(columns=['attraction_id', 'review_time','is_sub', 'count_id','tripduration'])
for item in filelist:
    medi=pd.read_csv(a+item)
    medi.columns=['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthday']
    medi['trip_time']=medi['stoptime']
    medi['trip_time']=trans_time(medi['trip_time'])
    medi['count_id']=1
    medi['is_sub']=medi['usertype'].apply(lambda x:1 if x=='Subscriber' else 0)
    medi_group=medi.groupby(['trip_time','to_station_id'])['is_sub','count_id','tripduration'].sum().reset_index()
    medi_group1=pd.merge(medi_group,dataq,left_on=['trip_time','to_station_id'],right_on=['review_time','recent_station'],how='inner')
    medix=medi_group1[['attraction_id', 'review_time','is_sub', 'count_id','tripduration']]
    medix['is_sub']=medix['is_sub']/medix['count_id']
    medix['is_sub']=medix['is_sub'].apply(lambda x:round(x,2))
    huizongz=pd.concat([huizongz,medix],axis=0)
huizongz.drop_duplicates(['attraction_id', 'review_time'],inplace=True)
huizongz.to_csv('D:\\python-document\\trips_to_attraction_zhongjianjieguo.csv',index=0)

aad=pd.read_csv('D:\\python-document\\tripad_attraction_station_(400,600,800).csv')
huizongz.drop_duplicates(['attraction_id', 'review_time'],inplace=True)
divvyas=pd.merge(aad,huizongz,on=['attraction_id', 'review_time'],how='left')
divvyas.to_csv('D:\\python-document\\tripad_attraction_station_(400,600,800)_divvy_trips.csv',index=0)
'''

F:\Adaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Divvy_Trips_2013.csv :完成
Divvy_Trips_2014-Q3-07.csv :完成
Divvy_Trips_2014-Q3-0809.csv :完成
Divvy_Trips_2014-Q4.csv :完成
Divvy_Trips_2014_Q1Q2.csv :完成
Divvy_Trips_2015-Q1.csv :完成
Divvy_Trips_2015-Q2.csv :完成
Divvy_Trips_2015_07.csv :完成
Divvy_Trips_2015_08.csv :完成
Divvy_Trips_2015_09.csv :完成
Divvy_Trips_2015_Q4.csv :完成
Divvy_Trips_2016_04.csv :完成
Divvy_Trips_2016_05.csv :完成
Divvy_Trips_2016_06.csv :完成
Divvy_Trips_2016_Q1.csv :完成
Divvy_Trips_2016_Q3.csv :完成
Divvy_Trips_2016_Q4.csv :完成
Divvy_Trips_2017_Q1.csv :完成
Divvy_Trips_2017_Q2.csv :完成
Divvy_Trips_2017_Q3.csv :完成
Divvy_Trips_2017_Q4.csv :完成
Divvy_Trips_2018_Q1.csv :完成
Divvy_Trips_2018_Q2.csv :完成
Divvy_Trips_2018_Q3.csv :完成
Divvy_Trips_2018_Q4.csv :完成


"\n#没有区分入站和出站点\nimport pandas as pd\n#读取文件名\nimport os\nfilelist=os.listdir('D:\\python-document\\divvy_trips\\xx\\')\na='D:\\python-document\\divvy_trips\\xx\\'\naad=pd.read_csv('D:\\python-document\\tripad_attraction_station_(400,600,800).csv')\ndataq=aad[['attraction_id','review_time','recent_station']]\nhuizongz=pd.DataFrame(columns=['attraction_id', 'review_time','is_sub', 'count_id','tripduration'])\nfor item in filelist:\n    medi=pd.read_csv(a+item)\n    medi.columns=['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',\n       'from_station_id', 'from_station_name', 'to_station_id',\n       'to_station_name', 'usertype', 'gender', 'birthday']\n    medi['trip_time']=medi['stoptime']\n    medi['trip_time']=trans_time(medi['trip_time'])\n    medi['count_id']=1\n    medi['is_sub']=medi['usertype'].apply(lambda x:1 if x=='Subscriber' else 0)\n    medi_group=medi.groupby(['trip_time','to_station_id'])['is_sub','count_id','tripduration'].sum().reset_index()\n    medi_group1=

In [5]:
#huizongz.columns

Index(['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthyear', 'trip_time',
       'is_sub', 'is_male', 'is_female', 'age'],
      dtype='object')

In [39]:
#订阅用户中也有一部分信息缺失，但可以忽略不计
#random.choices(population, weights=None, *, cum_weights=None, k=1),但没必要
#trips['tripduration'][trips.tripduration>12*3600]=12*3600
#trips.birthyear.notnull().sum()#13097298
#print(((trips.is_sub==1) & (trips.gender.isnull())).sum())#10313
#print(trips.is_sub.sum())#12974773

#(trips.birthyear.notnull()).sum()#13097298
#print(trips.usertype.value_counts())#Subscriber Customer Dependent
#print(trips.gender.value_counts())
#print(trips.birthyear.value_counts())

10313
12974773


 ### 合并trips数据（结果已保存）

In [65]:
#huizongz.columns

Index(['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthyear', 'trip_time',
       'is_sub', 'is_male', 'is_female', 'age'],
      dtype='object')

In [2]:
#区分入站和出站
#将入站与出站按时间合并到一个文件，也可以分开处理
import pandas as pd
def from_to():    
    huizongz=trips_origin(f)
#    huizongz=pd.read_csv("%s1.4.0trips_origin.csv"%filep)
    huizongz['count_id']=1
    medi_group_to=huizongz.groupby(['trip_time','to_station_id'])['is_sub','count_id', 'tripduration', 
                                                       'is_male','is_female','age'].sum().reset_index()
    medi_group_to.columns=['trip_time','to_station_id','to_is_sub','to_count_id','to_tripduration',
                              'to_is_male','to_is_female','to_age']
    medi_group_from=huizongz.groupby(['trip_time','from_station_id'])['is_sub','count_id', 'tripduration', 
                                                       'is_male','is_female','age'].sum().reset_index()
    medi_group_from.columns=['trip_time','from_station_id','from_is_sub','from_count_id','from_tripduration',
                                'from_is_male','from_is_female','from_age']

    medi_group_to.to_csv("%s1.4.1to_station.csv"%filep,index=False)
    medi_group_from.to_csv("%s1.4.1from_station.csv"%filep,index=False)
from_to()

'''
ss=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\to_station2.csv')
ss1=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\to_station1.csv')
ssx=pd.concat([ss,ss1],axis=0)
ssx.drop_duplicates(['trip_time','to_station_id'],inplace=True)
ssx.to_csv('D:\\python-document\\divvy_trips\\zh\\to_station3.csv')


dd=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\from_station2.csv')
dd1=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\from_station1.csv')
ddx=pd.concat([dd,dd1],axis=0)
ddx.drop_duplicates(['trip_time','from_station_id'],inplace=True)
ddx.to_csv('D:\\python-document\\divvy_trips\\zh\\from_station3.csv')
'''

F:\Adaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


"\nss=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\to_station2.csv')\nss1=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\to_station1.csv')\nssx=pd.concat([ss,ss1],axis=0)\nssx.drop_duplicates(['trip_time','to_station_id'],inplace=True)\nssx.to_csv('D:\\python-document\\divvy_trips\\zh\\to_station3.csv')\n\n\ndd=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\from_station2.csv')\ndd1=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\from_station1.csv')\nddx=pd.concat([dd,dd1],axis=0)\nddx.drop_duplicates(['trip_time','from_station_id'],inplace=True)\nddx.to_csv('D:\\python-document\\divvy_trips\\zh\\from_station3.csv')\n"

### 整合trips数据（按距离）

In [127]:
import numpy as np
import pandas as pd
def add_capa(d):
    datax=pd.read_csv("%s1.3.4_"%filep+str(d)+"m.csv")
    station=pd.read_csv("%s1.3.1station.csv"%filep)
#加上最近200m内总的容纳单车数
    sd=[]
    dis=[]
    for i in datax.index:
        ad=[]
        for j in eval(datax['station_id'][i]):
            cp=station.loc[station.id==j]['dpcapacity'].values[0]
            ad.append(cp)
        sd.append(sum(ad))
        dis.append(np.mean(eval(datax['distance'][i])))
    datax['dpcapacity']=pd.Series(sd)
    datax['mean_distance']=pd.Series(dis)
    datax['mean_distance']=datax['mean_distance'].apply(lambda x:round(x,0))
    datax.to_csv("%s1.4.2_"%filep+str(d)+'m+capacity.csv',index=0)
add_capa(200)
add_capa(300)

In [7]:
#计算200m的to_station
def to_station(d):
    m_200=pd.read_csv("%s1.4.2_"%filep+str(d)+'m+capacity.csv')
    to_station=pd.read_csv("%s1.4.1to_station.csv"%filep)
    is_sub=[]
    count=[]
    to_trip=[]
    to_male=[]
    to_female=[]
    to_age=[]
    for i in m_200.index:
        time=m_200['review_time'][i]
        subx=[]
        countx=[]
        tripx=[]
        malex=[]
        femalex=[]
        agex=[]
        for j in eval(m_200['station_id'][i]):
            try:
                #会出错的原因是空的series没有索引
                sub=to_station.loc[(to_station.trip_time==time )& (to_station.to_station_id==j)]['to_is_sub'].values[0]
                count1=to_station.loc[(to_station.trip_time==time )& (to_station.to_station_id== j)]['to_count_id'].values[0]
                trip=to_station.loc[(to_station.trip_time==time) & (to_station.to_station_id== j)]['to_tripduration'].values[0]
                male=to_station.loc[(to_station.trip_time==time) & (to_station.to_station_id== j)]['to_is_male'].values[0]
                female=to_station.loc[(to_station.trip_time==time) & (to_station.to_station_id== j)]['to_is_female'].values[0]
                age=to_station.loc[(to_station.trip_time==time) & (to_station.to_station_id== j)]['to_age'].values[0]
                subx.append(sub)
                countx.append(count1)
                tripx.append(trip) 
                malex.append(male)
                femalex.append(female)
                agex.append(age)
            except:
                print(i)
        is_sub.append(sum(subx))
        count.append(sum(countx))
        to_trip.append(sum(tripx))  
        to_male.append(sum(malex))
        to_female.append(sum(femalex))
        to_age.append(sum(agex))
    m_200['to_is_sub']=pd.Series(is_sub)
    m_200['to_counts']=pd.Series(count)
    m_200['to_tripduration']=pd.Series(to_trip)
    m_200['to_male']=pd.Series(to_male)
    m_200['to_female']=pd.Series(to_female)
    m_200['to_age']=pd.Series(to_age)
    m_200.to_csv("%s1.4.2_"%filep+str(d)+'m_to_station.csv',index=0)
to_station(200)
to_station(300)

65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
130
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
626
961
962
963
964
965
966
967
968
969
1054
1055
1056
1057
1058
1059
1060
1061
1062
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
1200
1201
1202
1203
1204
1205
1206
1207
1208
1209
1210
1358
1440
1441
1480
1480
1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494
1495
1496
1497
1498
1499
1500
1501
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1910
1911
1912
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2111
2112
2113
2135
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209

6375
6376
6377
6378
6379
6380
6381
6382
6383
6384
6385
6386
6387
6388
6389
6390
6433
6434
6435
6436
6437
6463
6464
6465
6466
6467
6468
6469
6470
6471
6472
6488
6489
6597
6597
6598
6599
6600
6601
6609
6610
6611
6612
6613
6614
6615
6616
6617
6648
6700
6700
6701
6701
6702
6702
6734
6735
6736
6737
6738
6739
6740
6741
6742
6743
6744
6745
6746
6747
6748
6749
6750
6751
6834
6835
6836
6837
6838
6839
6840
6880
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6945
6996
6997
6997
6998
6999
7000
7001
7002
7003
7004
7080
7081
7082
7083
7084
7085
7086
7087
7088
7089
7090
7117
7121
7138
7138
7139
7139
7140
7140
7141
7141
7142
7142
7143
7143
7144
7144
7145
7145
7161
7162
7163
7164
7165
7166
7167
7201
7272
7273
7274
7275
7276
7277
7278
7279
7280
7281
7308
7309
7310
7329
7330
7331
7332
7333
7334
7335
7336
7337
7338
7339
7340
7341
7376
7377
7435
7436
7504
7574
7575
7665
7666
7667
7668
7669
7670
7671
7672
7673
7674
7675
7676
7677
7730
7897
7957


In [9]:
#计算200m的from_station
def from_station(d):
    m_201=pd.read_csv("%s1.4.2_"%filep+str(d)+'m+capacity.csv')
    from_station=pd.read_csv("%s1.4.1from_station.csv"%filep)
    is_sub=[]
    count=[]
    to_trip=[]
    to_male=[]
    to_female=[]
    to_age=[]
    aaa=0
    for i in m_201.index:
        time=m_201['review_time'][i]
        subx=[]
        countx=[]
        tripx=[]
        malex=[]
        femalex=[]
        agex=[]
        
        for j in eval(m_201['station_id'][i]):
            try:
                sub=from_station.loc[(from_station.trip_time==time )& (from_station.from_station_id==j)]['from_is_sub'].values[0]
                count1=from_station.loc[(from_station.trip_time==time )& (from_station.from_station_id== j)]['from_count_id'].values[0]
                trip=from_station.loc[(from_station.trip_time==time) & (from_station.from_station_id== j)]['from_tripduration'].values[0]
                male=from_station.loc[(from_station.trip_time==time) & (from_station.from_station_id== j)]['from_is_male'].values[0]
                female=from_station.loc[(from_station.trip_time==time) & (from_station.from_station_id== j)]['from_is_female'].values[0]
                age=from_station.loc[(from_station.trip_time==time) & (from_station.from_station_id== j)]['from_age'].values[0]
                
                subx.append(sub)
                countx.append(count1)
                tripx.append(trip)
                malex.append(male)
                femalex.append(female)
                agex.append(age)
                
            except:
                aaa+=1
        is_sub.append(sum(subx))
        count.append(sum(countx))
        to_trip.append(sum(tripx)) 
        to_male.append(sum(malex))
        to_female.append(sum(femalex))
        to_age.append(sum(agex))
    print(aaa)
    m_201['from_is_sub']=pd.Series(is_sub)
    m_201['from_counts']=pd.Series(count)
    m_201['from_tripduration']=pd.Series(to_trip)
    m_201['from_male']=pd.Series(to_male)
    m_201['from_female']=pd.Series(to_female)
    m_201['from_age']=pd.Series(to_age)
    m_201.to_csv("%s1.4.2_"%filep+str(d)+'m_from_station.csv',index=0)
from_station(200)
from_station(300)    
#793 1647


793
1647


### 数据汇总

In [16]:
#处理to_station数据
def combine_trips(d):
    m_200=pd.read_csv("%s1.4.2_"%filep+str(d)+'m_to_station.csv')
    m_200['to_tripduration']=m_200['to_tripduration'].apply(lambda x:round(x/60,0))
    
    m_200['to_male'][m_200.to_is_sub!=0]=(m_200['to_male'][m_200.to_is_sub!=0]/m_200['to_is_sub']\
                                          [m_200.to_is_sub!=0]).apply(lambda x:round(x,2))
    m_200['to_male'][m_200.to_is_sub==0]=0
    
    m_200['to_female'][m_200.to_is_sub!=0]=(m_200['to_female'][m_200.to_is_sub!=0]/m_200['to_is_sub']\
                                            [m_200.to_is_sub!=0]).apply(lambda x:round(x,2))
    m_200['to_female'][m_200.to_is_sub==0]=0
    
    m_200['to_age'][m_200.to_is_sub!=0]=(m_200['to_age'][m_200.to_is_sub!=0]/m_200['to_is_sub']\
                                         [m_200.to_is_sub!=0]).apply(lambda x:round(x,2))
    m_200['to_age'][m_200.to_is_sub==0]=0
    
    m_200['to_is_sub']=(m_200['to_is_sub']/m_200['to_counts']).apply(lambda x:round(x,2))

    m_2001=m_200.loc[:,['attraction_id','review_time','dpcapacity','mean_distance','to_is_sub','to_counts','to_tripduration',
                       'to_male','to_female','to_age']]

#处理from_station数据
    m_201=pd.read_csv("%s1.4.2_"%filep+str(d)+'m_from_station.csv')
    m_201['from_tripduration']=m_201['from_tripduration'].apply(lambda x:round(x/60,0))
    
    m_201['from_male'][m_201.from_is_sub!=0]=(m_201['from_male'][m_201.from_is_sub!=0]/m_201['from_is_sub']\
                                               [m_201.from_is_sub!=0]).apply(lambda x:round(x,2))
    m_201['from_male'][m_201.from_is_sub==0]=0
    
    m_201['from_female'][m_201.from_is_sub!=0]=(m_201['from_female'][m_201.from_is_sub!=0]/m_201['from_is_sub']\
                                               [m_201.from_is_sub!=0]).apply(lambda x:round(x,2))
    m_201['from_female'][m_201.from_is_sub==0]=0
    
    m_201['from_age'][m_201.from_is_sub!=0]=(m_201['from_age'][m_201.from_is_sub!=0]/m_201['from_is_sub']\
                                               [m_201.from_is_sub!=0]).apply(lambda x:round(x,2))
    m_201['from_age'][m_201.from_is_sub==0]=0
    
    m_201['from_is_sub']=(m_201['from_is_sub']/m_201['from_counts']).apply(lambda x:round(x,2))
   
    m_2011=m_201.loc[:,['attraction_id','review_time','from_is_sub','from_counts','from_tripduration',
                       'from_male','from_female','from_age']]

#合并表格
    mrege=pd.read_csv('%s1.3.4_..+station'%filep+str(d)+'m.csv')
    mrege.review_time=trans_time(mrege.review_time)
    mda=pd.merge(mrege,m_2001,on=['attraction_id','review_time'],how='left')
    mda=pd.merge(mda,m_2011,on=['attraction_id','review_time'],how='left')

#加上总的行程数等特征,不知道有木有意义
    mda['trip_counts']=mda['to_counts']+mda['from_counts']
    mda['tripduration']=mda['from_tripduration']+mda['to_tripduration']
    mda['trip_per']=mda['from_counts']/mda['dpcapacity']
    mda['trip_per']=mda['trip_per'].apply(lambda x:round(x,2))
    mda.to_csv('%s1.4.3_..+station+trips'%filep+str(d)+'m.csv',index=0)
combine_trips(200)
combine_trips(300)

F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

## 整合犯罪数据（按社区）

### 计算社区中心（根据边界）

In [17]:
#加载社区数据
import pandas as pd
community=pd.read_csv('D:\\python-document\\control\\CommAreas.csv')
community.the_geom1=community.the_geom.apply(lambda x: x[12:])
community.the_geom2=community.the_geom1.apply(lambda x:x.strip().strip('()'))
community.the_geom2=community.the_geom2.apply(lambda x:list(x.split(',')))

F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """


In [18]:
#计算社区中心并保存
#处理社区数据
#这里取均值，而不是重心
import numpy as np
finalz=[]
for i in community.index:
    a=[]
    a=community.the_geom2[i]
    for j in range(len(a)):
        a[j]=a[j].strip().strip('()').split(' ')
        a[j][0].strip('()')
        a[j][1].strip('()')
    ab=np.array(a,dtype='float')
    q=ab.mean(axis=0)
    finalz.append(list(q))

finalz=pd.DataFrame(finalz,columns=['longitude','latitude'])
finalza=pd.concat([community,finalz],axis=1)
finalza=finalza[['AREA_NUMBE','longitude','latitude']]
finalza.to_csv('%s1.5.1commAreas.csv'%filep,index=0)

### 处理犯罪数据

In [34]:
#处理犯罪数据
def crime_data():
    cridata=pd.read_csv('D:\\python-document\\control\\Crimes_-_2001_to_present.csv')

    cridata1=cridata[['ID','Date','Primary Type','Latitude','Longitude', 'Location','Year','Community Area','Arrest']]
    cridata1['Year']=cridata1['Year'].apply(lambda x:str(x))
    cridata1=cridata1.query("Year>='2010' and Year<='2018'")
    cridata1['Date']=trans_time(cridata1['Date'])#时间有点长
    cridata1.to_csv('%s1.5.2crimedata.csv'%filep,index=0)
#根据犯罪类型来处理
    cridata1['total_crime_c']=1
    listx=cridata1['Primary Type'].value_counts().sort_values(ascending=False).index[:10]
    for i in listx:
        cridata1[i+'_c']=cridata1['Primary Type'].apply(lambda x:1 if x==i else 0)

    cridata2=cridata1.groupby(['Date','Community Area'])['total_crime_c', 'BATTERY_c',
       'OTHER OFFENSE_c', 'ROBBERY_c', 'NARCOTICS_c', 'THEFT_c',
       'CRIMINAL DAMAGE_c', 'ASSAULT_c', 'BURGLARY_c', 'DECEPTIVE PRACTICE_c',
       'MOTOR VEHICLE THEFT_c','Arrest'].sum() .reset_index()

    cridata2.to_csv('%s1.5.2crimedata_group.csv'%filep,index=0)

F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


### 汇总犯罪数据

In [4]:
attraction=pd.read_csv("%s1.2attraction.csv"%filep)
finalza=pd.read_csv('%s1.5.1commAreas.csv'%filep)
#计算距离景点最近的社区
from geopy.distance import geodesic
final=[]
for i in attraction.index: 
#    print(trip_attraction.loc[i].values[2])
    distance={}
    a=attraction['latitude'][i]
    b=attraction['longitude'][i]
    for x,y,z in zip(finalza['latitude'].values,finalza['longitude'].values,finalza['AREA_NUMBE'].values):
        dis=geodesic((a,b),(x,y)).m
        distance[z]=dis
    final.append(min(distance.items(), key=lambda x: x[1]))
print(len(final))#1954
finala=pd.DataFrame(final,columns=['community_number','distance_attr_commu'])
attraction=pd.concat([attraction,finala],axis=1)
attraction.to_csv("%s1.5.3comment_attraction_addCommArea.csv"%filep,index=0)

cridata2=pd.read_csv('%s1.5.2crimedata_group.csv'%filep)
cridata_attra=pd.merge(cridata2,attraction,left_on='Community Area',right_on='community_number',how='inner')
cridata_attra=cridata_attra.loc[:,['Date','community_number','total_crime_c',
                                   'BATTERY_c', 'OTHER OFFENSE_c', 'ROBBERY_c', 'NARCOTICS_c',
                                   'THEFT_c','CRIMINAL DAMAGE_c', 'ASSAULT_c', 'BURGLARY_c', 
                                   'DECEPTIVE PRACTICE_c','MOTOR VEHICLE THEFT_c', 'id','Arrest']]
def hebing(d):
    zhiqian=pd.read_csv('%s1.4.3_..+station+trips'%filep+str(d)+'m.csv')
    zhiqian=pd.merge(zhiqian,cridata_attra,left_on=['attraction_id','review_time'],
                 right_on=[ 'id','Date'],how='left')
    zhiqian1=zhiqian.query("review_time>'2009-12'")
    zhiqian1.to_csv('%s1.5.3huizong.csv'%filep,index=0)
hebing(200)
hebing(300)

1954


# 数据预处理与特征提取

## 季度变量

In [ ]:
#接下来做季度变量
trip_attraction1['review_time']=trip_attraction1['review_time'].apply(lambda x:str(x.year)+'-'+'{:02}'.format(x.month))
def mapx(a):
    x1=['01','02','03']
    x2=['04','05','06']
    x3=['07','08','09']
    if a in x1:
        return 'Q1'
    elif a in x2:
        return 'Q2'
    elif a in x3:
        return 'Q3'
    else:
        return 'Q4'
trip_attraction1['quarter']=trip_attraction1['review_time'].apply(lambda x:mapx(x[-2:]))
trip_attraction1['year_quarter']=trip_attraction1['review_time'].apply(lambda x:x[0:4]+mapx(x[-2:]))
trip_attraction2=trip_attraction1.drop('Unnamed: 0.1',axis=1)
trip_attraction2.columns
trip_attraction2.to_csv('d://python-document//tripad_attraction_station.csv',index=0)#不保留行索引更舒服


In [ ]:
'2018-06-12'>'2018-06-06'